In [6]:
# Import the quantum chemistry driver and related units
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

# Import the qubit mapper library for Ci+ and Ci
from qiskit_nature.second_q.mappers import JordanWignerMapper, BravyiKitaevMapper, BravyiKitaevSuperFastMapper, ParityMapper

# Import a classical analyzer to compare results with VQE
from qiskit_algorithms import NumPyMinimumEigensolver

# Import VQE and related libraries
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP, NELDER_MEAD
from qiskit.primitives import Estimator, StatevectorEstimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

# Import ground state solver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

import numpy as np

# For plotting
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
driver = PySCFDriver(
    atom="O 0 0 0.115; H 0 0.754 -0.459; H 0 -0.754 -0.459",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

full_problem = driver.run()
fc_transformer = FreezeCoreTransformer()
es_problem = fc_transformer.transform(full_problem)
print(full_problem.num_particles, full_problem.num_spatial_orbitals)
print(es_problem.num_particles, es_problem.num_spatial_orbitals)

(5, 5) 7
(4, 4) 6


In [8]:
mapper = ParityMapper()

In [9]:
ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [10]:
calc = GroundStateEigensolver(mapper, vqe_solver)

In [11]:
numpy_solver = NumPyMinimumEigensolver()
calc2 = GroundStateEigensolver(mapper, numpy_solver)

In [12]:
%%time

res2 = calc2.solve(es_problem)
print(res2)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -84.294760857849
  - computed part:      -23.61038438923
  - FreezeCoreTransformer extracted energy part: -60.684376468619
~ Nuclear repulsion energy (Hartree): 9.285714221678
> Total ground state energy (Hartree): -75.009046636171
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 8.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.00377945]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.644391947386]
    - computed part:      [0.0  0.0  0.210650346165]
    - FreezeCoreTransformer extracted energy part: [0.0  0.0  0.43374160122]
  > Dipole moment (a.u.): [0.0  0.0  -0.640612497386]  Total: 0.640612497386
                 (debye): [0.0  0.0  -1.628274400796]  Total: 1.628274400796
 
CPU times: user 3.82 s, sys: 81.1 ms, total: 3.91 s
Wall time: 3.36 s


In [ ]:
%%time
res = calc.solve(es_problem)
print(res)